# 2023 - 2024 Expenses Project! need to figure out how to get this file onto github

In this project, I collected a years worth of grocery shopping receipts from the 2022 - 2023 academic school year (August through June). From there, I manually inputted all of the data into an excel file, which I will...

1. read the excel file into this jupyter notebook file via pandas
2. do cleaning and analysis on the excel file
3. and produce visualizations of the data

As this is my first project, the goal is to practice creating a readable data-analysis file that is up to the industry standard, showcase my skills in data cleaning, and perform some initial exploratory analysis, and produce visualizations of that data. In subsequent projects, the goal will be to build on these skills and utilize machine learning algorithms to make price predictions.

In [134]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
from scipy import stats

First, we will read in the file using pandas.

In [136]:
os.chdir("C:/Users/AVILA/Desktop")
base_path = os.getcwd()
file_name = "Expenses23-24.csv"
file_location = os.path.join(base_path, file_name)

file_location
df = pd.read_csv(file_location)
data = df


Next, we will conduct some exploratory analysis. Since we know that the data has not been cleaned yet, it is helpful to look through the data to observe any irregularities before conducting any formal analysis or cleaning.

In [138]:
# first look into the data, trying to figure out what will be necessary steps to clean
data.columns = data.columns.str.lower()
data["good"] = data["good"].str.lower()
data.columns
data["good"].head(20)
data["good"].unique
data.shape

#needed to create a column to capture units, as price was sometimes defined as the sum of
# multiple identical units.
data["units purchased"] = [1 for i in data["good"]] 

#needed to create a column to capture the weight of each unit, ex. apples come in 4lb sacks.
# default unit is in lbs. 
data["weight"] = 1

# repeatedly used the following code to check various values.
data[data["good"].str.contains("kefit")]["good"]

314    yogurt kefit plain low fat 
Name: good, dtype: object

# Data Cleaning

We found that there were instances in the data, particularly relating to bananas, where a price listed was actually the sum of multiple units purchased at the same time. The following code corrects those instances in the data and produces a column that indicates the number of units purchased, as well as the per-unit-price for each item.

In [141]:
#below is a series of individual changes to the dataset for a given unit. 
# data sometimes contained units purchased in the name of the good (ex. cucumbers x 3), so made 
# individual adjustments to include the units purchased into the dataset
bananas = [6,8,8,6,6,5,6,6,6,7,5,6,7,6,7,5,6,6,6,7,5,5,1,6,6,4]
positions = [24,43,55,66,91,107,122,124,146,160,187,207,224,237,251,292,321,330,368,390,400,412,451,467,511,516]

for i in range(26):
    data.iloc[positions[i], 5] = bananas[i]
    
peeled_tomatos = [112, 114, 261, 262]
for i in range(4):
    data.loc[i, "weight"] = .625

C:\Users\AVILA\AppData\Local\Temp\ipykernel_1340\796793536.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.625' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, "weight"] = .625


In looking through the names column in the data, we found a significant number of irregularities regarding naming conventions. The following cells are an attempt to standardize those naming conventions using a series of dictionaries. The cell immediately following this one contains those dictionaries.

In [143]:
#below is a series of dictionaries to standardize the naming convention of items in the dataset.
#dictionaries are then used to map to standardized values.
milk_dictionary =                   {"2% mlik": "2% Gallon, Milk", 
                                     "milk gallon low fat 2%": "2% Gallon, Milk",
                                     "milk gallon low fat 2% ": "2% Gallon, Milk",
                                     "milk gallong low fat 2%": "2% Gallon, Milk"}
roasted_unsalted_peanuts =          {"roasted and unsalted peanuts":"bag peanuts, roasted, unsalted",
                                     "roasted unsalted peanuts": "bag peanuts, roasted, unsalted"}
fifty_percent_less_salt =           {"50% less salt roasted peanuts": "bag peanuts, 50% less salt"}
lemon_bag_twolbs =                  {"lemon bag 2 lbs": "lemon bag, 2lbs",
                                     "a lemon bag 2 lb": "lemon bag, 2lbs",
                                     "lemons seedless bag 2l" : "lemon bag, 2lbs",
                                     "lemons seedless bag 2lb": "lemon bag, 2lbs"}
lemon_individual =                   {"lemon each x 4": "lemon",
                                      "lemon each": "lemon",
                                      "lemon each x 2": "lemon"}
almond_butter =                     {"almond butter crunch no stir": "almond butter",
                                     "almond butter, crunchy": "almond butter",
                                     "almong butter creamy no stir": "almond butter"}
conventional_bananas =              {"banana, conventional 8": "bananas, conventional",
                                     "banana, conventional x 8": "bananas, conventional",
                                     "banana, conventional x 6": "bananas, conventional",
                                     "banana each x 5": "bananas, conventional",
                                     "banana each x 6": "bananas, conventional",
                                     "banana each x 7": "bananas, conventional",
                                     "banana conventional x 6": "bananas, conventional",
                                     "bananas conventional each x 7": "bananas, conventional",
                                     "banana conventional each x 6": "bananas, conventional",
                                     "banana conventional each x 7": "bananas, conventional",
                                     'banana conventional each x 5': "bananas, conventional",
                                     "banana each": "bananas, conventional",
                                     "banana conventional each x 4": "bananas, conventional",
                                     "banana each conventional x 6": "bananas, conventional",
                                     "banan each x 5": "bananas, conventional",
                                     "banans conventional each x 6": "bananas, conventional"}
organic_bananas =                   {"banana organic each x 6": "banana, organic",
                                     "6 bananas, organic": "banana, organic"}
gala_apples =                       {"apple pouch gala 4 lb": "apple pouch, gala"}
pineapple =                         {"pineapple each": "pineapple"}
honey_crisp =                       {"apple pouch honey crisp": "apple pouch, honey crisp"}
bacon =                             {"bacon abf uncured dry fu": "bacon"}
rice_vinegar =                      {"vinegar rice": "rice vinegar"}
balsamic_vinegar =                  {"balsamic vinegar of mode": "balsamic vinegar"}
apple_cider_vinegar =               {"vinegar organic raw apple cider": "organic raw apple cider vinegar"}
red_wine_vinegar =                  {"red wine vinegar organic": "organic red wine vinegar"}
broccoli =                          {"broccoli bunch organic": "broccoli, organic",
                                     "broccoli bunch organic each": "broccoli, organic"}
brusselsprouts =                    {"brussels sprouts 1lb": "brussels sprouts"}
bustelo =                           {"cafe bustelo gr esp[resso": "cafe bustelo pack"}
gooseberries =                      {"gooseberries 6oz": "gooseberries",
                                     "cape gooseberries 6oz": "gooseberries"}
carrots =                           {"carrots whole org 1lb": "organic carrots, bag",
                                     "carrots whole org 1 lb": "organic carrots, bag",
                                     "carrots whole organic 1 lb": "organic carrots, bag"}
celery =                            {"celery hearts 2 ct": "celery",
                                     "celery hearts organic": "celery"}
cleaning_tools =                    {"cealning tools": "cleaning tools"}
anchovies =                         {"ceno anchovies in olive oil x 2": "ceno anchovies"}
san_marzano =                       {"cento san marzano peeled 10 oz": "can san marzano tomatos",
                                     "cento san marzano peeled tomatos": "can san marzano tomatos"}
cottage_cheese =                    {"cottage cheese quart who": "cottage cheese",
                                     "cottage cheese quarrt who" : "cottage cheese"}
cucumbers_hothouse =                {"cucumbers hothouse": "cucumbers, hothouse",
                                     "cucumbers": "cucumbers, hothouse",
                                     "cucumbers hothouse each": "cucumbers, hothouse"}
cucumbers_persian =                 {"cucumbers persian 1 lb": "cucumbers, persian",
                                     "cucumbers persian1 lb": "cucumbers, persian"}
eggs =                              {"eggs, large white dozen": "eggs, dozen",
                                     "eggs large white dozen": "eggs, dozen"}
el_milagro =                        {"el milagro flour tortillas": "flour tortillas, el milagro",
                                     "el milagro flour tortillas ": "flour tortillas, el milagro",
                                     "el milagro tortillas": "flour tortillas, el milagro",
                                     "el milagro flour tortillas x 2": "flour tortillas, el milagro"}
mushroom =                          {"white button mushrooms,": "mushrooms, white button",
                                     "mushrooms white button": "mushrooms, white button",
                                     "mushrooms white botton": "mushrooms, white button",
                                     "mushrooms white button": "mushrooms, white button",
                                     "mushrooms white button organic": "mushrooms, white button",
                                     "white button mushrooms, ": "mushrooms, white button"}
strawberries =                      {"fzn strawberries": "strawberries, frozen",
                                     "frozen strawberries": "strawberries, frozen"}
garlic =                            {"garlic each": "garlic",
                                     "garlic eac": "garlic",
                                     "garlic ": "garlic"}
org_garlic =                        {"galirc organic 3 oz": "garlic pouch, organic"}
shaving_cream =                     {"gilette foamy shaving cream 11 oz": "shaving cream",
                                     "gillette foamy shae cream 11 oz": "shaving cream"}
ginger =                            {"ginger organic 3oz": "ginger, organic",
                                     "ginger organic 3 oz": "ginger, organic"}
green_onions =                      {"green onions 6 oz": "green onions",
                                     "green onions 6oz": "green onions"}
kale_greens =                       {"greens kale 10 oz": "kale greens",
                                     "greens kale 10oz": "kale greens"}
dill =                              {"herb dill 1 oz": "herb dill",
                                     "herb dill 1oz": "herb dill"}
homestyle_tortillas =               {"home style tortilla": "trader joes tortillas",
                                     "homestyle flour tortillas": "trader joes tortillas",
                                     "homestyle flour tortilla": "trader joes tortillas",
                                     "homestyle flourtortillas": "trader joes tortillas",
                                     "homestyle tortillas": "trader joes tortillas",
                                     "homestyle flourtortilla": "trader joes tortillas"}
red_onion =                         {"red onion jumbo each": "red onion jumbo",
                                     "nions red jubmo each x 2": "red onion jumbo",
                                     "onions red jumbo each":"red onion jumbo"}
sesame =                            {"oil rganic toasted sesame": "sesame oil",
                                     "oil organic sesame oil":"sesame oil"}
onion =                             {"onions jumbo yellow each": "yellow onion",
                                     "onions sweet jumbo each" : "yellow onion",
                                     "onions jumbo yellow": "yellow onion",
                                     "onions sweet jumbo each x 2 ": "yellow onion",
                                     "onions jumbo yellow": "yellow onion"}
persimmons =                        {"persimmons x 6": "fuyu persimmons",
                                     "persimmons each fuyu x 6": "fuyu persimmons",
                                     "persimmons each fuyu": "fuyu persimmons"}
raisins =                           {"seedless raisin": "seedless raisins",
                                     "thompson seedless raisins": "seedless raisins"}
shallots =                          {"shallot, each": "shallot",
                                     "shallot each": "shallot",
                                     "shallots" : "shallot",
                                     "shallots each x 7": "shallot",
                                     "shallots eacah": "shallot"}
tomato_medley =                     {"tomatoes medly 1lb": "tomato medley",
                                     "tomatoes medly 1 lb": "tomato medley",
                                     "tomtaoes medly 1 lb": "tomato medley",
                                     "tomatos medley 1 lb": "tomato medley",
                                     "tomateos medly 1 lb": "tomato medley",
                                     "tomatoes medley 1 lb": "tomato medley",
                                     "tomatos medly 1 lb": "tomato medley"}
roma =                              {"tomatos roma each x 3": "roma tomato",
                                     "tomatoes roma each x 3": "roma tomato"}

The following cell utilizes these dictionaries to replace all incorrect values with a standardized naming convention for each grocery item. Additionally, we produce a list of the names(key_list) to identify if there are any duplicates, missing values, or anything out of the ordinary in the naming conventions.

In [145]:
for i in dictionary_list_goods:
    data["good"] = data["good"].replace(i)

# cleaning naming convention data
good_groupby = data.groupby("good")
good_groupby.groups
key_list = [key for key in good_groupby.groups]

In working with the name column, we identified a series of one off changes that were required. These changes are made in the cell block below. Additionally, as we moved on into prices, there were one off changes that occured which we needed to change at the data-cell level. All of these changes are listed below.

In [147]:
# below are a series of individual adjustments to the dataset to either standardize naming conventions
# or to standardize/convert one off values

data.loc[332, "weight"] =   .375
data.loc[474, "weight"] =   .375
data.iloc[376, 5] = 4
data.iloc[476, 5] = 2
data.iloc[497, 5] = 2
data.iloc[187, 5]
data.iloc[148, 6] = 4
data.iloc[419, 6] = 4
data.loc[503, "units purchased"] = 2
data.loc[336, "units purchased"] = 2
data.loc[261, "units purchased"] = 2
data.loc[505, "weight"] = .375
data.loc[454, "good"] = "herb cilantro bunch"
data.loc[210, "weight"] = .6875
data.loc[440, "weight"] = .6875
data.loc[95, "weight"] = .1875
data.loc[244, "weight"] = .1875
data.loc [253, "weight"] = .1875
data.loc[456, "weight"] = .1875
data.loc[280, "weight"] = .375
data.loc[120, "weight"] = .625
data.loc[141, "weight"] = .625
data.loc[255, 'weight'] = .625
data.loc[470, "weight"] = .625
data.loc[421, "good"] = "greens kale tuscan organic"
data.loc[165, "good"] = "hash browns"
data.loc[243, "weight"] = .0625
data.loc[329, "weight"] = .0625
data.loc[334, "units purchased"] = 5
data.loc[334, "good"] = "jalapeno"
data.loc[275, "good"] = "natural boneless skinless chicken thighs"
data.loc[381, "good"] = "natural boneless skinles chicken thighs"
data.loc[399, "units purchased"] = 2
data.loc[346, "units purchased"] = 2
data.loc[488, "good"] = "pasta gnocchi"
data.loc[153, "units purchased"] = 6
data.loc[408, "units purchased"] = 6
data.loc[299, "good"] = 'potato bag teeny tiny'
data.loc[290, "units purchased"] = 2
data.loc[339, "units purchased"] = 3
data.loc[504, "weight"] = .375
data.loc[504, "good"] = "salad organic baby spinach"
data.loc[278, "weight"] = .3125
data.loc[357, "weight"] = .3125
data.loc[333, "units purchased"] = 7
data.loc[465, "good"] = "shredded mexican blend"
data.loc[212, "good"] = "unbleached all purpose flour"
data.loc[208, "units purchased"] = 3
data.loc[472, "units purchased"] = 3
data.loc[524, "good"] = "up and up"
data.loc[314, "good"] = "yogurt kefir plain low fat"

We now follow a similar process to standardize the store names. First we identify if there are misspelled or incorrect store names, then change them via dictionaries and the replace method.

In [149]:
#cleaning/standardizing store names columns
data["store name"].unique()

traderjoes = {"Trdaer Joes": "Trader Joes",
              "Tradre Joes": "Trader Joes",
              "Traderr Joes": "Trader Joes",
              "Trader Joses": "Trader Joes",
              "Trader Joes ": "Trader Joes"}
walgreens = {"Wallgreens": "Walgreens",
             "Wallgreens ": "Wallgreens"}
cvs = {"CVS pharmacy": "CVS Pharmacy"}

dictionary_list_stores = [traderjoes, walgreens, cvs]
for i in dictionary_list_stores:
    data["store name"] = data["store name"].replace(i)

Next, we identified that there were commas and periods in the price column indicating decimal values. The following cell standardizes those values to all periods.

In [151]:
#the following code identifies commas in the price column, removes the commas,
# and replaces the data in the original dataset

type(data.loc[1, "price"])

b = pd.DataFrame(data[data["price"].str.contains(",")]["price"])
b = b.reset_index()

for i in range(6):
    b.loc[i, "price"] = b.loc[i, "price"].replace(",", ".")
    data.loc[b.loc[i, "index"], "price"] = b.loc[i, "price"]

data["price"] = data["price"].astype(float)

Next, we transform the date column to datetime objects.

In [153]:
#turning purchase date to datetime object
type(data.loc[1, "purchase date"])
data.loc[:, "purchase date"] = pd.to_datetime(data["purchase date"], format = "%m/%d/%Y", errors = "coerce")

Finally, we create the price-per-unit column

In [155]:
#calculating price per unit column
data.loc[:, "price per unit"] = data["price"]/data["units purchased"]

# Data Pre Analysis

In [157]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   timestamp        530 non-null    object 
 1   good             530 non-null    object 
 2   price            530 non-null    float64
 3   store name       530 non-null    object 
 4   purchase date    526 non-null    object 
 5   units purchased  530 non-null    int64  
 6   weight           530 non-null    float64
 7   price per unit   530 non-null    float64
dtypes: float64(3), int64(1), object(4)
memory usage: 33.3+ KB


In [158]:
data.describe()

,price,units purchased,weight,price per unit
count,530.000000,530.000000,530.000000,530.000000
mean,37.432223,1.307547,0.986321,37.320336
std,489.942849,1.180232,0.223894,489.950879
min,0.190000,1.000000,0.062500,0.190000
25%,1.965000,1.000000,1.000000,1.890000
50%,2.990000,1.000000,1.000000,2.990000
75%,3.990000,1.000000,1.000000,3.990000
max,8343.000000,8.000000,4.000000,8343.000000


We wanted to check what the highest values were in price column, so we sorted the values using the following code.

In [172]:
highest_values = data.sort_values(by = "price", ascending = False)
highest_values

,timestamp,good,price,store name,purchase date,units purchased,weight,price per unit
381,6/7/2024 11:17:56,natural boneless skinles chicken thighs,8343.000,Trader Joes,2024-02-26 00:00:00,1,1.000,8343.000
364,6/7/2024 11:13:35,multi floral and clover,7349.000,Trader Joes,2023-11-26 00:00:00,1,1.000,7349.000
33,5/30/2024 20:34:21,la preferida dry black beans,2019.000,Hyde Park Produce,2023-09-21 00:00:00,1,1.000,2019.000
2,5/30/2024 20:20:04,jbl flip 6,129.990,Target,2024-04-26 00:00:00,1,0.625,129.990
10,5/30/2024 20:24:33,folding table,59.990,Ace Hardware,2024-04-27 00:00:00,1,1.000,59.990
...,...,...,...,...,...,...,...,...
177,5/31/2024 7:58:57,"ginger, loose 2.99/lb",0.390,Hyde Park Produce,2024-01-05 00:00:00,1,1.000,0.390
337,6/7/2024 11:05:29,ginger loose .1 lb @ 2.99 / lb,0.300,Hyde Park Produce,2024-01-26 00:00:00,1,1.000,0.300
65,5/30/2024 20:49:19,pepper jalapeno each,0.290,Trader Joes,2024-04-11 00:00:00,1,1.000,0.290
454,6/7/2024 11:57:11,herb cilantro bunch,0.199,Trader Joes,2023-07-09 00:00:00,1,1.000,0.199


Upon inspection, it is very clear that there are 3 extremely large outliers, which do not correspond with values that make sense for the corresponding goods they represented Ex. 8343$ for chicken thighs. Therefore it makes sense to  remove these values and treat them as outliers, which we do in the cell below. 

In [179]:
# found that the only outliers were extremely large values greater than ~130.
data = data[data["price"] < 200]

Running the describe code again gives us the following values, which we believe to be more representative of the true paraemeters of the dataset.

In [182]:
data.describe()

,price,units purchased,weight,price per unit
count,527.000000,527.000000,527.000000,527.000000
mean,4.038099,1.309298,0.986243,3.925575
std,7.229254,1.183364,0.224529,7.255754
min,0.190000,1.000000,0.062500,0.190000
25%,1.925000,1.000000,1.000000,1.890000
50%,2.990000,1.000000,1.000000,2.990000
75%,3.990000,1.000000,1.000000,3.790000
max,129.990000,8.000000,4.000000,129.990000


In [123]:
for key, value in good_groupby.groups:
    if len(value > 2):
        new_good_groupby = {key:value for key,value in good_groupby.groups}

# now we start with the analysis
for i in good_groupby.groups:
    price = [data.loc[y, "price per unit"] for y in good_groupby.groups[i]]
    date = [data.loc[x, "purchase date"] for x in good_groupby.groups[i]]
    sns.relplot(x = date, y = price)

ValueError: too many values to unpack (expected 2)